In [1]:
from functions import *
from torch.optim import Adam

In [2]:
max_ = -1
for k,v in mapping.items():
    if v>max_:
        k_max = k
        max_ = v

In [4]:
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_lookup,len(k_max))

2.1029411764705874

In [4]:
max_

2.1029411764705874

In [2]:
from functions_pt import *

In [3]:
embed_size = 128
dropout = 0.1
multiple_factor = 2
layers = 2
batch_size = 64
agg = 'max' # mean,sum,min,max
model = pointNet(layers,embed_size,agg,dropout,multiple_factor)#.to('cuda')
#model.load_state_dict(torch.load('baseline_noBN.pt'))
#model.eval()
model = model.to('cuda')

In [4]:
words_embed = np.load('possible_words_embed.npy')

In [8]:
# slightly worse than # of steps required by mapping as expected
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_model,len(k_max),model=model,words_embed=words_embed,top=0.6)

2.1249999999999987

In [2]:
# entropy policy
for i in np.linspace(0,7,15):
    steps = evaluate(matrix,np.arange(2309),policy_entropy,2309,factor=i)
    print('factor:{}, steps:{}'.format(i,steps))

factor:0.0, steps:3.5275010827197937
factor:0.5, steps:3.4638371589432673
factor:1.0, steps:3.4634040710264204
factor:1.5, steps:3.4634040710264204
factor:2.0, steps:3.4634040710264204
factor:2.5, steps:3.462970983109573
factor:3.0, steps:3.4638371589432673
factor:3.5, steps:3.4642702468601136


KeyboardInterrupt: 

In [2]:
# entropy policy with depth info
for i in np.linspace(1,2,5):
    steps = evaluate_depth(matrix,np.arange(2309),policy_entropy_depth,2309,factor=i,depth=1)
    print('factor:{}, steps:{}'.format(i,steps))

factor:1.0, steps:3.4634040710264204
factor:1.25, steps:3.462970983109573
factor:1.5, steps:3.4638371589432673
factor:1.75, steps:3.4642702468601136
factor:2.0, steps:3.4655695106106554


In [5]:
# see how well model does on full problem that it is not trained on
#     if count == 1: return 1
#     if count == 2: return 1.5
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.5266349068860987

RL Loop

In [ ]:
# model trained from start
# top = 0.5 does not work better
# layers = 3, size = 64 about the same results
# value += eps * best * np.random.randn() and eps=0.16/(i+1) and from trained model
lr = 2e-5
clip = 0.06
epochs = 2
damp = 1
iterations = 20
max_data = 2000
states = set()
for i in range(iterations):
    # policy evaluation
    model.eval()
    score = evaluate_save(matrix,np.arange(2309),policy_model,2309,0,model=model,words_embed=words_embed,top=0.6,
                         eps=0.1/(i+1))
    print('iter:{}, score:{}'.format(i,score))
    
    # prepare data for training
    index,val,log_prob = zip(*out)
    for element in index:
        states.add(tuple(element))
    print('set size:{}'.format(len(states)))
    prob = np.array(log_prob)
    prob -= prob.mean()
    prob = np.exp(prob/damp)
    prob = torch.tensor(prob,dtype=torch.float32,device='cuda')
    length = torch.tensor([len(i) for i in index],dtype=torch.float32,device='cuda')
    words = torch.tensor(words_embed[np.concatenate(index)]).long().to('cuda')
    ys = torch.tensor(val,dtype=torch.float32,device='cuda')

    
    # policy update
    model.train()
    opt = Adam(model.parameters(),lr=lr)
    for _ in range(epochs):
        yhat = model((words,length))
        loss = torch.sum(prob*(ys-yhat)**2)/torch.sum(prob)
        loss.backward()
        print('loss:{}'.format(loss.item()))
        clip_grad_value_(model.parameters(),clip)
        opt.step()
        opt.zero_grad()
    #print(model.w)
    while len(out)>max_data:
        out.pop(0)

iter:0, score:3.839757470766566
set size:529
loss:4.430811405181885
loss:3.9819977283477783
iter:1, score:3.824599393676918
set size:1053
loss:3.6125292778015137
loss:3.068514585494995
iter:2, score:3.8315288003464723
set size:1489
loss:2.3788745403289795
loss:1.7603644132614136
iter:3, score:3.8215677782589874
set size:1878
loss:1.3503117561340332
loss:0.9525684714317322
